In [ ]:
import os
cwd=os.getcwd()
os.chdir(os.path.dirname(cwd))
print(os.getcwd())
import json
from core.data_ingestion import crunchbase
import dotenv
dotenv.load_dotenv()


%load_ext autoreload
%autoreload 2

raw_crunchbase_data=json.loads(open('raw_crunchbase_data.json').read())

In [3]:
import psycopg2
conn = psycopg2.connect(
    dbname="startup_database",
    user="postgres",
    password=os.getenv("DB_PASSWORD"),
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [6]:
# get all records
cur.execute("SELECT * FROM crunchbase;")
crunchbase = cur.fetchall()

# get all industries
cur.execute("SELECT * FROM industry_connections;")
industry_connections = cur.fetchall()

In [5]:
"""Get the counts for where the best companies come from"""
cur.execute("""
WITH relevant_companies AS (
    SELECT DISTINCT c.primary_key, c.cb_rank, c.company_name, c.last_funding_type, c.employees
    FROM crunchbase c
    LEFT JOIN industry_connections ic ON c.primary_key = ic.crunchbase_primary_key
    WHERE ic.industry_name = 'Automation'
        OR c.last_funding_type = 'Seed'
),
all_companies AS (
    SELECT *, 
            ROW_NUMBER() OVER (ORDER BY cb_rank ASC) AS rank_within_set
    FROM relevant_companies
),
combined_data AS (
    SELECT 
        ac.company_name,
        ac.last_funding_type,
        ic.industry_name,
        ac.cb_rank,
        CASE WHEN ac.rank_within_set <= 200 THEN 1 ELSE 0 END AS is_top_200,
        CASE WHEN ic.industry_name = 'Automation' THEN 1 ELSE 0 END AS is_automation,
        CASE WHEN ac.last_funding_type = 'Seed' THEN 1 ELSE 0 END AS is_seed_funding,
        CASE WHEN ac.last_funding_type = 'Early Stage Venture' THEN 1 ELSE 0 END AS is_venture_funding,
        CASE WHEN ac.employees = 101 then 1 else 0 end as is_large_company,
        ROW_NUMBER() OVER (PARTITION BY ac.company_name ORDER BY 
            CASE WHEN ic.industry_name = 'Automation' THEN 0 ELSE 1 END, 
            ic.industry_name
        ) AS row_num
    FROM all_companies ac
    LEFT JOIN industry_connections ic ON ac.primary_key = ic.crunchbase_primary_key
),
deduplicated_data AS (
    SELECT *
    FROM combined_data
    WHERE row_num = 1
)
SELECT 
    company_name,
    last_funding_type,
    industry_name,
    SUM(is_top_200) OVER () AS total_top_200,
    SUM(is_automation) OVER () AS total_automation,
    SUM(is_seed_funding) OVER () AS total_seed_funding,
    SUM(is_venture_funding) OVER () AS total_venture_funding,
    SUM(is_large_company) OVER () AS total_large_companies

FROM deduplicated_data
ORDER BY cb_rank ASC NULLS LAST
""")
records=cur.fetchall()

In [ ]:
print(records[0])
print(len(records))

In [109]:
"""Returns top 200 crunchbase, automation industries, and seed funding companies"""

funding_search_values=["'Series A'", "'Series B'","'Series C'", "'Angel'", "'Post-IPO Equity'", "'Seed'"]
fuding_search_string=" OR last_funding_type=".join(funding_search_values)

sql_query = """WITH top_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type, employees
    FROM crunchbase
    ORDER BY cb_rank
    LIMIT 200
)
SELECT tc.primary_key, tc.company_name, tc.last_funding_type, ic.industry_name
FROM industry_connections ic
LEFT JOIN top_companies tc ON ic.crunchbase_primary_key = tc.primary_key
WHERE tc.primary_key IS NOT NULL
AND ic.industry_name = 'Automation'
OR tc.last_funding_type = {fuding_search_string}
OR (tc.employees > 10 AND tc.cb_rank
ORDER BY tc.cb_rank DESC


""".format(fuding_search_string=fuding_search_string)

sql_query = """WITH top_200_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type
    FROM crunchbase
    ORDER BY cb_rank
    LIMIT 200
),
seed_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type
    FROM crunchbase
    WHERE last_funding_type = 'Seed'
),
large_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type
    FROM crunchbase
    WHERE employees > 10
),
automation_companies AS (
    SELECT DISTINCT c.primary_key, c.cb_rank, c.company_name, c.last_funding_type
    FROM crunchbase c
    JOIN industry_connections ic ON c.primary_key = ic.crunchbase_primary_key
    WHERE ic.industry_name = 'Automations'
),
chemical_companies AS (
    SELECT DISTINCT c.primary_key
    FROM crunchbase c
    JOIN industry_connections ic ON c.primary_key = ic.crunchbase_primary_key
    WHERE ic.industry_name = 'Chemical'
)
SELECT DISTINCT c.*
FROM (
    SELECT * FROM top_200_companies
    UNION
    SELECT * FROM seed_companies
    UNION
    SELECT * FROM large_companies
    UNION
    SELECT * FROM automation_companies
) c
WHERE c.primary_key NOT IN (SELECT primary_key FROM chemical_companies)
"""

sql_query = """WITH top_200_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type
    FROM crunchbase
    ORDER BY cb_rank
    LIMIT 200
),
seed_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type
    FROM crunchbase
    WHERE last_funding_type = {fuding_search_string}
),
large_companies AS (
    SELECT primary_key, cb_rank, company_name, last_funding_type
    FROM crunchbase
    WHERE (employees > 10 AND cb_rank <100000)
),
automation_companies AS (
    SELECT DISTINCT c.primary_key, c.cb_rank, c.company_name, c.last_funding_type
    FROM crunchbase c
    JOIN industry_connections ic ON c.primary_key = ic.crunchbase_primary_key
    WHERE ic.industry_name = 'Automation'
),
chemical_companies AS (
    SELECT DISTINCT c.primary_key
    FROM crunchbase c
    JOIN industry_connections ic ON c.primary_key = ic.crunchbase_primary_key
    WHERE ic.industry_name = 'Chemical'
)
SELECT DISTINCT c.*
FROM (
    SELECT * FROM top_200_companies
    UNION
    SELECT * FROM large_companies
    UNION
    SELECT * FROM automation_companies
    UNION
    SELECT * FROM seed_companies

) c
WHERE c.primary_key NOT IN (SELECT primary_key FROM chemical_companies)
""".format(fuding_search_string=fuding_search_string)

cur.execute(sql_query)
records = cur.fetchall()

In [ ]:
    UNION
    SELECT * FROM seed_companies
    UNION
    SELECT * FROM large_companies
    UNION
    SELECT * FROM automation_companies

# Includes
- the top 200 companies when sorted by crunchbase.cb_rank (in the database not by cb_rank value, in otherwords not by cb_rank<200 but instead by the top 200 in the dataset when sorted)
- all companies with crunchbase.last_funding_type = 'Seed'
- all companies with crunchbase.employees > 10
- all companies with industry_connections.industry_name='Automations'

# Excludes
- all companies with industry_connections.industry_name='Chemical'

the returned data should duplicate no crunchbase tables


# Table context
- there are no duplicates in the crunchbase table
- A cruncbase company can have many industries connected to it so long as (crunchbase.primary_key=industry_connections.crunchbase_primary_key), this means that there are many industry associations with any company

In [106]:
cur.execute("ROLLBACK")

In [ ]:
print(records[0])
print(records[1])
print(records[2])
print(records[3])
# print(records[4])
print(records[-1])
print(len(records))

In [ ]:
print(records[0])
print(records[1])
print(records[2])
print(records[3])
# print(records[4])
print(records[-1])
print(len(records))

# all data VIEW

In [ ]:
c=0
for company in crunchbase:
    p_key=company[0]
    for industry_connection in industry_connections:
        if industry_connection[0]==p_key:
            if industry_connection[1]=='Automation':
                c+=1
                break
print(c)

In [ ]:
crunchbase[0][9]

In [ ]:
c=0
types=set()
for company in crunchbase:
    p_key=company[0]
    types.add(company[9])
    if company[9]=="Series B":
        c+=1
    # for industry_connection in industry_connections:
    #     if industry_connection[0]==p_key:
    #         if industry_connection[1]=='Automation':
    #             c+=1
    #             break
print(c)

In [ ]:
crucnh

In [ ]:
print(list(types))

In [ ]:
'Series A', 'Series B','Series C', 'Angel', 'Post-IPO Equity', 'Seed'

In [99]:
cur.execute("""
SELECT * FROM crunchbase
ORDER BY cb_rank ASC
""")
sorted_companies=cur.fetchall()

In [ ]:
sorted_companies[500]